In [ ]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))

import torch

from accelerate import notebook_launcher

from utils.data_utils import BrainDataset, process_file_last
from utils.train_utils import TrainConfig, run_train_model, count_parameters, load_model_weights

from dataclasses import dataclass
from simple_parsing.helpers import Serializable

import albumentations as A

import matplotlib.pyplot as plt
import gc

In [ ]:
from models.brain_mae import EncoderConfig, Encoder, MAE, MAEConfig

### Init models

In [ ]:
encoder_config = EncoderConfig()
mae_config = MAEConfig()
model = MAE(encoder_config=encoder_config, mae_config=mae_config)


In [ ]:
window_size = encoder_config.window_size
n_electrodes = 256 * 4
max_tokens = 25

train_transform = A.Compose([
    A.RandomCrop(height=window_size, width=n_electrodes, always_apply=True),
])

test_transform = A.Compose([
    A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
                  border_mode=0, value=0, always_apply=True),
    A.Crop(x_min=0, x_max=n_electrodes, y_min=0, y_max=window_size, always_apply=True)
])

data_path = Path("/drive/data/competitionData")

tokenize_function = None
process_file_function = process_file_last

train_dataset = BrainDataset(data_path / 'train', 
                             process_file_function=process_file_function, 
                             tokenize_function=tokenize_function, 
                             transform=train_transform, 
                             max_tokens=max_tokens)

gc.collect()
test_dataset = BrainDataset(data_path / 'test', 
                            process_file_function=process_file_function, 
                            tokenize_function=tokenize_function, 
                            transform=test_transform, 
                            max_tokens=max_tokens)


submit_dataset = BrainDataset(data_path / 'competitionHoldOut', 
                            process_file_function=process_file_function, 
                            tokenize_function=tokenize_function, 
                            transform=test_transform, 
                            max_tokens=max_tokens)

gc.collect()


train_dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset, submit_dataset])
test_dataset = submit_dataset
print(len(train_dataset))

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

x, y, date = next(iter(test_dataloader))
print(x.shape, y.shape, date.shape)

loss, logits = model(x, y, date)


### Load datasets

In [ ]:
project_name = 'simple_mae'
save_folder = Path("/drive/logs/kovalev")

train_config = TrainConfig(exp_name='brain-mae-full-datasets',
                           mixed_precision=True,
                           batch_size=256, 
                           grad_accum=8,
                           num_workers=3,
                           pin_memory=True, 
                           eval_interval=250, 
                           learning_rate=1e-4,
                           weight_decay=0, 
                           grad_clip=10,
                           lr_decay_iters=20_000, 
                           warmup_iters=1000, 
                           project_name=project_name, 
                           save_folder=save_folder
                          )

model = torch.compile(model)

train_dataset = torch.utils.data.Subset(train_dataset, torch.arange(256))
args = (model, (train_dataset, test_dataset), train_config, encoder_config)
notebook_launcher(run_train_model, args, num_processes=1)

In [ ]:
all_features = []
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

for x, y, date in test_dataloader:
    x = x.to('cuda')
    date = date.to('cuda')
    features = model.encoder(x, date_info=date)
    all_features.extend(features)

In [ ]:
def visualize_data(x, recon, binary_mask):
    x_masked_input = binary_mask * x
    x_unmasked_input = ~binary_mask * x

    recon_masked = binary_mask * recon
    recon_unmasked = ~binary_mask * recon


    t_st, t_end = 64, 128
    c_st, c_end = 0, 32

    fig, ax = plt.subplots(3,2, figsize = (16, 16))
    ax[0, 0].imshow(x[0, t_st:t_end, c_st:c_end].T, vmax=1)
    ax[1, 0].imshow(x_unmasked_input[0, t_st:t_end, c_st:c_end].T, vmax=1)
    ax[2, 0].imshow(x_masked_input[0, t_st:t_end, c_st:c_end].T, vmax=1)

    ax[0, 1].imshow(recon[0, t_st:t_end, c_st:c_end].T, vmax=1)
    ax[1, 1].imshow(recon_unmasked[0, t_st:t_end, c_st:c_end].T, vmax=1)
    ax[2, 1].imshow(recon_masked[0, t_st:t_end, c_st:c_end].T, vmax=1)
